## Exercise 1

Poisson regression is a Generalized Linear Model, used to model count data. It takes the form

$$\mathbb{E}(\mu|x)=\exp(w_1\,x_1+\ldots+w_k\,x_k+b),$$

where the observed counts $y$ are drawn from a Poisson distribution on the expected counts: 

$$y_i \sim \text{Poisson}(\mu_i).$$

1. Download and import Load the smoking dataset from: [https://data.princeton.edu/wws509/datasets/#smoking](https://data.princeton.edu/wws509/datasets/#smoking). Then perform a train-test split on the data;


In [204]:
%reset -f
import numpy as np
import pandas as pd

import pyro
import pyro.distributions as dist
import pyro.optim as optim

from pyro.infer import SVI, Trace_ELBO
from pyro.infer import Predictive

import torch
import torch.distributions.constraints as constraints

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

pyro.set_rng_seed(42)

In [205]:
data = pd.read_csv("https://data.princeton.edu/wws509/datasets/smoking.raw",sep="\t",header=None)
data.columns = ["Age_Group","Smoking_Status","Population","Deaths"]
data.head()

,Age_Group,Smoking_Status,Population,Deaths
0,1,1,656,18
1,2,1,359,22
2,3,1,249,19
3,4,1,632,55
4,5,1,1067,117


* age at the start of follow-up: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
* smoking status: coded 1 = never smoked, 2 = smoked cigars or pipe only, 3 = smoked cigarettes and cigar or pipe, and 4 = smoked cigarettes only,
* population: number of male pensioners followed, and
* deaths: number of deaths in a six-year period.

In [206]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Age_Group       36 non-null     int64
 1   Smoking_Status  36 non-null     int64
 2   Population      36 non-null     int64
 3   Deaths          36 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


Now we perform a train-test split on the data:
- **train data** - 80% of the observations will be used to perfom inference on our model
- **test data** - the remaining 20% will be used for testing the correctness of posterior predictions 

In [207]:
scaler = MinMaxScaler()
data["Population"] = scaler.fit_transform(pd.DataFrame(data["Population"]))

In [208]:
deaths = torch.tensor(data["Deaths"].values, dtype=torch.float)
predictors = torch.stack([torch.tensor(data[column].values,dtype=torch.float) 
                            for column in data.columns if column != "Deaths"],1)

X_train, X_test, y_train, y_test = train_test_split(predictors, deaths, test_size=0.20, 
                                                    random_state=42,shuffle=True)

print("X_train.shape =", X_train.shape,"\ny_train.shape =", y_train.shape)
print("\nX_test.shape =", X_test.shape,"\ny_test.shape =", y_test.shape)

X_train.shape = torch.Size([28, 3]) 
y_train.shape = torch.Size([28])

X_test.shape = torch.Size([8, 3]) 
y_test.shape = torch.Size([8])


2. Fit a Poisson bayesian regression model using the number of deaths as the response variable and the other columns as the explanatory variables;

The target variable in our model is the number of deaths and we wish to infer the parameters corresponding to the following predictors

$$
\text{Deaths}=\text{exp}(w_0\cdot\text{Age_Group}+w_1\cdot\text{Smoking_Status}+w_2\cdot\text{Population}+b) + \epsilon
$$

We set a normal prior on $w$, a Log-Normal on the bias term $b$ and a uniformly distributed std for the gaussian noise on $\hat{y}$

\begin{align*}
w&\sim\mathcal{N}(0,1)\\
b&\sim\text{LogNormal}(0,1)\\
\hat{\mu}&= \text{exp}(w x+ b) \\
y &\sim \text{Poisson}(\hat{\mu}).
\end{align*}

Then we define the family of posterior distributions, by setting a Gamma distribution on $w$ and a Log-Normal on $b$, and run SVI inference on $(x,y)$ data.

Notice the prior distribution on the bias term makes this regression problem analytically intractable.

**will be read**

https://towardsdatascience.com/an-illustrated-guide-to-the-poisson-regression-model-50cccba15958

https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Poisson_Regression.pdf

**Poisson ile ilgili okuma yapılacak, tanım aralıklarına bakılacak**

**w  için negatif değer bulunmaması gerekiyor ona bakılacak, muhati nasıl pozitif yaparım** 

In [209]:
n_observations, n_predictors = predictors.shape

# sample weights
w = pyro.sample("w", dist.Normal(torch.zeros(n_predictors), 
                                    torch.ones(n_predictors)))
b = pyro.sample("b", dist.LogNormal(torch.zeros(1), torch.ones(1)))

mu_hat = torch.exp((w*predictors).sum(dim=1) + b)

# condition on the observations
with pyro.plate("deaths", len(deaths)):
    pyro.sample("obs", dist.Poisson(mu_hat), obs=deaths)

In [210]:
pyro.clear_param_store()
def death_model(predictors, deaths):
    
    n_observations, n_predictors = predictors.shape
    
    # sample weights
    w = pyro.sample("w", dist.Normal(torch.zeros(n_predictors), 
                                        torch.ones(n_predictors)))
    b = pyro.sample("b", dist.LogNormal(torch.zeros(1), torch.ones(1)))
    
    mu_hat = torch.exp((w*predictors).sum(dim=1) + b)
    
    # condition on the observations
    with pyro.plate("deaths", len(deaths)):
        pyro.sample("obs", dist.Poisson(mu_hat), obs=deaths)
        
def death_guide(predictors, deaths=None):
    
    n_observations, n_predictors = predictors.shape
        
    w_loc = pyro.param("w_loc", torch.rand(n_predictors), constraint=constraints.positive)
    w_scale = pyro.param("w_scale", torch.rand(n_predictors), 
                         constraint=constraints.positive)
    
    w = pyro.sample("w", dist.Gamma(w_loc, w_scale))
    
    b_loc = pyro.param("b_loc", torch.rand(1))
    b_scale = pyro.param("b_scale", torch.rand(1), constraint=constraints.positive)
    
    b = pyro.sample("b", dist.LogNormal(b_loc, b_scale))
    
death_svi = SVI(model=death_model, guide=death_guide, 
              optim=optim.ClippedAdam({'lr' : 0.01}), 
              loss=Trace_ELBO())

for step in range(2000):
    loss = death_svi.step(X_train, y_train)/len(X_train)
    if step % 100 == 0:
        print(f"Step {step} : loss = {loss}")


Step 0 : loss = 2.6560565079145475e+26
Step 100 : loss = 564.9201554385002
Step 200 : loss = 708.6234738565753
Step 300 : loss = 278.42650453445816
Step 400 : loss = 222461873.94874647
Step 500 : loss = 216.57358952759532
Step 600 : loss = 1136982.402666029
Step 700 : loss = 863.1879060592859
Step 800 : loss = 194.9383085622035
Step 900 : loss = 927.9817657068543
Step 1000 : loss = 45951328.3234046
Step 1100 : loss = 103987967.24565305
Step 1200 : loss = 48849.89831843372
Step 1300 : loss = 189.87693486920108
Step 1400 : loss = 30770.749232856742
Step 1500 : loss = 218976.99306298795
Step 1600 : loss = 278.81435853329475
Step 1700 : loss = 661.209050953894
Step 1800 : loss = 508.6218641709407
Step 1900 : loss = 57.804438422715556


In [211]:
print("Inferred params:", list(pyro.get_param_store().keys()), end="\n\n")

# w_i and b posterior mean
inferred_w = pyro.get_param_store()["w_loc"]
inferred_b = pyro.get_param_store()["b_loc"]

for i,w in enumerate(inferred_w):
    print(f"w_{i} = {w.item():.8f}")
print(f"b = {inferred_b.item():.8f}")

Inferred params: ['w_loc', 'w_scale', 'b_loc', 'b_scale']

w_0 = 0.42066144
w_1 = 0.33679302
w_2 = 0.65289156
b = -0.27868899


**Posterior predictive distribution**

We can use the `Predictive` utility class, corresponding to the posterior predictive distribution, to evaluate our model on test data. Here we compute some summary statistics (mean, std and qualtiles) on $100$ samples from the posterior predictive:

In [212]:
# print latent params quantile information
def summary(samples):
    stats = {}
    for par_name, values in samples.items():
        marginal = pd.DataFrame(values)
        percentiles=[.05, 0.5, 0.95]
        describe = marginal.describe(percentiles).transpose()
        stats[par_name] = describe[["mean", "std", "5%", "50%", "95%"]]
    return stats

# define the posterior predictive
predictive = Predictive(model=death_model, guide=death_guide, num_samples=100,
                        return_sites=("w","b"))

# get posterior samples on test data
svi_samples = {k: v.detach().numpy() for k, v in predictive(X_test, y_test).items()}

# show summary statistics
for key, value in summary(svi_samples).items():
    print(f"Sampled parameter = {key}\n\n{value}\n")

Sampled parameter = w

       mean       std        5%       50%       95%
0  0.179517  0.265638  0.001096  0.057790  0.709780
1  1.465643  2.859591  0.000348  0.374409  6.264905
2  3.089101  4.225276  0.055791  1.360811  9.302667

Sampled parameter = b

       mean       std        5%       50%      95%
0  0.815523  0.287393  0.444661  0.754776  1.26463




3. Evaluate the regression fit on test data using MAE and MSE error metrics.

The most known metrics for comparing different regression models are the **Mean Absolute Error** (MAE)

$$\frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|$$

and the **Mean Squared Error** (MSE)

$$\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2,$$

where $n$ is the number of observations, $y$ are the true values `y_test` and $\hat{y}$ are the predicted values `y_pred`.

In [213]:
# compute predictions using the inferred paramters
y_pred = torch.exp((inferred_w * X_test).sum(1)) + inferred_b

print("MAE =", torch.nn.L1Loss()(y_test, y_pred).item())
print("MSE =", torch.nn.MSELoss()(y_test, y_pred).item())

MAE = 280.67466154190674
MSE = 141635.45808262352


## Exercise 2

The Iris dataset contains petal and sepal length and width for three different types of Iris flowers: Setosa, Versicolour, and Virginica.





3. Evaluate your bayesian classifier on test data: compute the overall test accuracy and class-wise accuracy for the three different flower categories.

1. Import the Iris dataset from `sklearn`:
```
from sklearn import datasets
iris = datasets.load_iris()
```
and perform a train-test split on the data.

In [214]:
from sklearn import datasets
iris = datasets.load_iris()

In [215]:
iris_data = torch.from_numpy(iris.data)
iris_target = torch.from_numpy(iris.target)
X_train, X_test, y_train, y_test = train_test_split(iris_data, iris_target, test_size=0.20,
                                                    random_state=42,shuffle=True)

print("x_train.shape =", X_train.shape,"\ny_train.shape =", y_train.shape)
print("\nx_test.shape =", X_test.shape,"\ny_test.shape =", y_test.shape)

x_train.shape = torch.Size([120, 4]) 
y_train.shape = torch.Size([120])

x_test.shape = torch.Size([30, 4]) 
y_test.shape = torch.Size([30])


2. Fit a multinomial bayesian logistic regression model on the four predictors petal length/width and sepal length/width. 

In [216]:
def log_reg_model(X, y):
    n_observations, n_predictors = X.shape
    
    w = pyro.sample("w", dist.Normal(torch.zeros(n_predictors), torch.ones(n_predictors)))
    b = pyro.sample("b", dist.Normal(0.,1.))
    # non-linearity
    yhat = torch.sigmoid((w*X).sum(dim=1) + b)
    
    with pyro.plate("data", n_observations):
        # sampling 0-1 labels from Bernoulli distribution
        y = pyro.sample("y", dist.Bernoulli(yhat), obs=y)
        
def log_reg_guide(X, y=None):
    
    n_observations, n_predictors = X.shape
    
    w_loc = pyro.param("w_loc", torch.rand(n_predictors))
    w_scale = pyro.param("w_scale", torch.rand(n_predictors),constraint=constraints.positive)
    w = pyro.sample("w", dist.Normal(w_loc, w_scale))
    
    b_loc = pyro.param("b_loc", torch.rand(1))
    b_scale = pyro.param("b_scale", torch.rand(1),constraint=constraints.positive)
    b = pyro.sample("b", dist.Normal(b_loc, b_scale))
    print(w,b)
    


In [217]:
log_reg_svi = SVI(model=log_reg_model, guide=log_reg_guide, 
              optim=optim.ClippedAdam({'lr' : 0.0002}), 
              loss=Trace_ELBO()) 

losses = []
for step in range(10000):
    loss = log_reg_svi.step(X_train, y_train)
    losses.append(loss)
    if step % 1000 == 0:
        print(f"Step {step} : loss = {loss}")
        
fig, ax = plt.subplots(figsize=figsize)
ax.plot(losses)
ax.set_title("ELBO loss");

tensor([4.0799, 0.4217, 0.7353], grad_fn=<AddBackward0>) tensor([-0.8557], grad_fn=<AddBackward0>)


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1
Trace Shapes:    
 Param Sites:    
Sample Sites:    
       w dist 4 |
        value 3 |
       b dist   |
        value 1 |